In [1]:
import xarray as xr
import iris
import mule
from dask.distributed import Client
import numpy as np
import cftime

In [2]:
nc_prefix="/scratch/ly62/dr4292/experiments/flood22-continuous/netcdf"
um_prefix="/scratch/ly62/dr4292/cylc-run/u-cs142-waci-longrun/share/cycle"

In [3]:
ts="20220221T1200Z"
fn="umnsaa_pvera000"
tol=1.e-6
eps=1.e-15

In [4]:
def printwarn(cube,da,msg):
    print(f"Warning: [{cube.name()} <-> {da.name}]: {msg}")

In [5]:
def printerr(cube,da,msg):
    print(f"ERROR: [{cube.name()} <-> {da.name}]: {msg}")

In [6]:
def coord_translate(coord):
    if coord.startswith("model_rho_level_number"): return "model_level_number"
    if coord.startswith("model_theta_level_number"): return "model_level_number"
    if coord.startswith("lat") : return "latitude"
    if coord.startswith("lon"): return "longitude"
    if coord.startswith("sigma"): return "sigma"
    if coord.startswith("rho_level_height"): return "level_height"
    if coord.startswith("theta_level_height"): return "level_height"
    if coord.startswith("time"): return "time"
    if coord.startswith("height"): return "height"
    return coord

In [7]:
def float_arr_compare(a,b):
    return (abs(a-b) <= np.maximum(tol*np.maximum(abs(a),abs(b)),eps) ).all()

In [8]:
def check_coords(cube,dataarray):
    cube_coords_seen=[]
    #for da_coord in dataarray.dims:
    for da_coord in dataarray.coords:
        cube_coord=coord_translate(da_coord)
        ### The data arrays have more coordinates than the cubes, as um2netcdf4 sticks
        ### every scalar coord to every field. So if the coords are scalar, we can skip
        ### them if they're not present.
        try:
            cube_coord_data=cube.coord(cube_coord).points
        except iris.exceptions.CoordinateNotFoundError:
            if dataarray.coords[da_coord].size != 1:
                printerr(cube,dataarray,f"Coordinate {cube_coord} translated from {da_coord} not present in original UM data or")
                printerr(cube,dataarray,f"Multiple coordinates matching {cube_coord}, translated from {da_coord} found")
            else:
                printwarn(cube,dataarray,f"Scalar coord {da_coord} missing from original UM data")
            continue
        cube_coords_seen.append(cube_coord)
        
        if da_coord.startswith("time"):
            ### Convert to cftime
            try:
                dataarray_coord=dataarray.convert_calendar("gregorian",dim=da_coord,use_cftime=True).coords[da_coord]
            except ValueError:
                ### Something dodgy about time_2
                dataarray_coord=dataarray.coords[da_coord]
        else:
            dataarray_coord=dataarray.coords[da_coord]
        
        if cube_coord=="time":
            all(np.array([ cube.coord(cube_coord).units.num2date(i) for i in cube_coord_data],dtype="object")==dataarray_coord)
        elif cube_coord_data.dtype == "float":
            #if not all(abs(cube_coord_data-dataarray_coord.values) < tol):
            if cube_coord=="pressure":
                ### This gets reversed:
                cube_coord_data=np.flipud(100.0*cube_coord_data)
            if not float_arr_compare(cube_coord_data,dataarray_coord.values):
                ### Are there any other coords that might match this one?
                coord_mismatch=[ True, ]
                for c in dataarray.coords:
                    if c.startswith(cube_coord) and c != dataarray_coord:
                        coord_mismatch.append(not float_arr_compare(cube_coord_data,dataarray.coords[c].values) )
                        if not coord_mismatch[-1]:
                            printwarn(cube,dataarray,f"Scalar coord {cube_coord} derived from {da_coord} matches da coord of different name: {c}")
                if all(coord_mismatch):
                    printerr(f"Difference between equivalent coordinates {cube_coord} and {da_coord} exceeds tolerance {tol}")
        else:
            if not all(cube_coord_data == dataarray_coord.values):
                ### Are there any other coords that might match this one?
                coord_mismatch=[ True, ]
                for c in dataarray.coords:
                    if c.startswith(cube_coord) and c != dataarray_coord:
                        coord_mismatch.append(not all(cube_coord_data == dataarray.coords[c].values) )
                if all(coord_mismatch):
                    printerr(f"Difference between equivalent integer coordinates {cube_coord} and {da_coord}")
            
    for name in [ i.name() for i in cube.coords() ]:
        if name == "forecast_period": continue
        if name == "forecast_reference_time": continue
        if name not in cube_coords_seen:
            printerr(f"Coordinate {name} not found in converted netCDF field")

In [9]:
def check_field(cube,dataarray):
    if cube.data.dtype in ("float","float32"):
        #if not (abs(cube.data - dataarray.data) <= abs(cube.data*tol)).all():
        if cube.coords()[0].name()=="pseudo_level":
            ### Dims are going to be in the wrong order - transpose the dataarray
            da_dat=dataarray.transpose('pseudo_level',*[i for i in dataarray.dims if i != "pseudo_level" ])
        elif "pressure" in [ c.name() for c in cube.coords() ]:
            da_dat=dataarray.isel(pressure=slice(None,None,-1)).data
        else:
            da_dat=dataarray.data
        if not float_arr_compare(cube.data,da_dat):
            printerr(f"Field values {cube.name()} and {dataarray.name} differ by larger than tolerance")
    else:
        if not (cube.data == dataarray.data).all():
            printerr(f"Field values {cube.name()} and {dataarray.name} differ")

In [10]:
def get_ncfld(cube,ds):
    ### First guess...
    ncfld_name="fld_"+str(cube.attributes['STASH'])[3:]
    ### Is there a cell method?
    try:
        cube_method=cube.cell_methods[0].method
    except:
        ### Nope, lets be on our way
        cube_method = None
    ### Does the cube method match the xarray method?
    try:
        da_method=ds[ncfld_name].cell_methods.split(' ')[1]
    except:
        ### No cell method, we're done
        da_method = None
    
    if cube_method==da_method:
        return ds[ncfld_name]

    ### If we make it here, the cell methods don't match.
    ### Make a list of all fields in the dataset with a matching stash code
    for da in [ ds[i] for i in ds if i.startswith(ncfld_name) ]:
        try:
            da_method=da.cell_methods.split(' ')[1]
        except:
            continue
        if da_method == cube_method:
            return da
    
    return None

In [11]:
umf=iris.load(f'{um_prefix}/{ts}/aus2200/d0198/RA3/um/{fn}')
umf

[<iris 'Cube' of m01s01i202 / (unknown) (time: 7; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of Turbulent mixing height after boundary layer / (m) (time: 7; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of m01s03i253 / (unknown) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of Cumulus capped boundary layer indicator / (1) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of air_temperature / (K) (time: 7; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of atmosphere_boundary_layer_thickness / (m) (time: 2; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of dew_point_temperature / (K) (time: 7; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of fog_area_fraction / (1) (time: 7; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of land_binary_mask / (1) (latitude: 2120; longitude: 2600)>,
<iris 'Cube' of relative_humidity / (%) (time: 7; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of sea_ice_area_fraction / (1) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of snowfall_amount / (kg m-2) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of surface_air_pressure / (Pa) (time: 2; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of surface_altitude / (m) (latitude: 2120; longitude: 2600)>,
<iris 'Cube' of surface_downwelling_longwave_flux_in_air / (W m-2) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of surface_downwelling_shortwave_flux_in_air / (W m-2) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of surface_net_downward_longwave_flux / (W m-2) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of surface_temperature / (K) (time: 7; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of surface_upward_latent_heat_flux / (W m-2) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of surface_upward_sensible_heat_flux / (W m-2) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of toa_incoming_shortwave_flux / (W m-2) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of toa_outgoing_longwave_flux / (W m-2) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of toa_outgoing_shortwave_flux / (W m-2) (time: 7; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of toa_outgoing_shortwave_flux / (W m-2) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of visibility_in_air / (m) (time: 7; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of wind_speed_of_gust / (m s-1) (time: 6; latitude: 2120; longitude: 2600)>,
<iris 'Cube' of x_wind / (m s-1) (time: 7; latitude: 2121; longitude: 2600)>,
<iris 'Cube' of y_wind / (m s-1) (time: 7; latitude: 2121; longitude: 2600)>]

In [12]:
ds=xr.open_dataset(f'{nc_prefix}/{ts}/aus2200/d0198/RA3/um/{fn}.nc')
ds

<xarray.Dataset>
Dimensions:             (time: 6, lat: 2120, lon: 2600, bnds: 2, time_0: 7,
                         time_1: 2, time_2: 1, time_3: 7, time_4: 6,
                         lat_0: 2121, lon_0: 2600, time_5: 6)
Coordinates: (12/13)
  * time                (time) datetime64[ns] 2022-02-21T13:00:00 ... 2022-02...
  * lat                 (lat) float64 -48.79 -48.77 -48.75 ... -6.852 -6.832
  * lon                 (lon) float64 107.5 107.5 107.6 ... 158.9 159.0 159.0
  * time_0              (time_0) datetime64[ns] 2022-02-21T12:00:00 ... 2022-...
  * time_1              (time_1) datetime64[ns] 2022-02-21T15:00:00 2022-02-2...
  * time_2              (time_2) datetime64[ns] 2022-02-21T12:00:00
    ...                  ...
  * time_4              (time_4) datetime64[ns] 2022-02-21T12:00:30 ... 2022-...
  * lat_0               (lat_0) float64 -48.8 -48.78 -48.76 ... -6.842 -6.822
  * lon_0               (lon_0) float64 107.5 107.5 107.5 ... 158.9 159.0 159.0
    height              float64 ...
    height_0            float64 ...
  * time_5              (time_5) datetime64[ns] 2022-02-21T12:30:00 ... 2022-...
Dimensions without coordinates: bnds
Data variables: (12/34)
    fld_s00i023         (time, lat, lon) float32 ...
    latitude_longitude  int32 ...
    lat_bnds            (lat, bnds) float64 ...
    lon_bnds            (lon, bnds) float64 ...
    fld_s00i024         (time_0, lat, lon) float32 ...
    fld_s00i025         (time_1, lat, lon) float32 ...
    ...                  ...
    fld_s03i253         (time, lat, lon) float32 ...
    fld_s03i281         (time_3, lat, lon) float32 ...
    fld_s03i304         (time_3, lat, lon) float32 ...
    fld_s03i310         (time, lat, lon) float32 ...
    fld_s03i463_max     (time_5, lat, lon) float32 ...
    time_5_bnds         (time_5, bnds) datetime64[ns] ...
Attributes:
    history:      File /scratch/ly62/dr4292/cylc-run/u-cs142-waci-longrun/sha...
    Conventions:  CF-1.6
    source:       Data from Met Office Unified Model
    um_version:   12.2

In [13]:
%%time
seen_nc_fields=[]
for cube_field in umf:
    #ncfld_name="fld_"+str(cube_field.attributes['STASH'])[3:]
    ncfld=get_ncfld(cube_field,ds)
    print(f"Comparing UM output {cube_field.name()} and netCDF output {ncfld.name}")
    seen_nc_fields.append(ncfld.name)
    try:
        check_coords(cube_field,ncfld)
        #for coord in ncfld.coords:
            #if 'level_number' in coord:
                #check_field(cube_field,ncfld.chunk({coord:3}))
                #break
        #else:
        check_field(cube_field,ncfld)
    except KeyError:
        #print(f"Error! UM field {cube_field.name()} not found in netCDF output")
        raise
for name in [ i for i in ds if i.startswith('fld_') ]:
    if name not in seen_nc_fields:
        print(f"Error! netCDF field {name} not found in UM output")

Comparing UM output m01s01i202 and netCDF output fld_s01i202
Comparing UM output Turbulent mixing height after boundary layer and netCDF output fld_s03i304
Comparing UM output m01s03i253 and netCDF output fld_s03i253
Comparing UM output Cumulus capped boundary layer indicator and netCDF output fld_s03i310
Comparing UM output air_temperature and netCDF output fld_s03i236
Comparing UM output atmosphere_boundary_layer_thickness and netCDF output fld_s00i025
Comparing UM output dew_point_temperature and netCDF output fld_s03i250
Comparing UM output fog_area_fraction and netCDF output fld_s03i248
Comparing UM output land_binary_mask and netCDF output fld_s00i030
Comparing UM output relative_humidity and netCDF output fld_s03i245
Comparing UM output sea_ice_area_fraction and netCDF output fld_s00i031
Comparing UM output snowfall_amount and netCDF output fld_s00i023
Comparing UM output surface_air_pressure and netCDF output fld_s00i409
Comparing UM output surface_altitude and netCDF output fl